In [108]:
import json, os, re
from lxml import html

In [109]:
def merge_content(folder):
    filelist = []

    for root, dirs, files in os.walk(folder):
        for filename in files:
            if filename[0] == 's':
                filelist.append(root+"/"+filename)

    filelist.sort()
    content = []
    
    for filename in filelist:
        with open(filename, encoding="utf8") as file:
            content.append("[[[FILE: " +filename+"]]]")
            start = 0
            for line in file:
                if "<!-- eh -->" in line or "<!-- ex -->" in line: start = 1
                if "<!-- sf -->" in line or "<!-- sy -->" in line: start = 0
                if start == 1: content.append(line)
    
    return(content)

In [110]:
def get_date(folder):
    session = str(int(folder.split("/")[1].split("schuz")[0]))
    with open(folder+"/"+session+"-1.htm", encoding="utf8") as file:
        for line in file:
            if line.startswith("<title>"): 
                return line.split("schůze, ")[1].split("</title>")[0].replace("&nbsp;", " ")

In [115]:
def proc_merged(folder):
    content = merge_content(folder)
    date = get_date(folder)
    
    output = []
    speaking = ""
    link = ""
    curspeech = {}
    for line in content:
        line = line.replace('&nbsp;',' ')
        line = line.replace('***','')
        
        # link: http://www.psp.cz/eknih/2010ps/stenprot/007schuz/s007080.htm
        if "[[[FILE: " in line:
            innerlink = line.split("[[[FILE: ")[1].split("]]]")[0].split("/")
            link = "http://www.psp.cz/eknih/" + innerlink[0] + "ps/stenprot/" + innerlink[1] + "/" + innerlink[2]
            line = re.sub("\[\[\[FILE:.*\]\]\]", "", line)
    
        # speech:
        if "<b><a href=" in line or "<b><a name=" in line:
            if speaking: 
                output.append(curspeech)
                curspeech = {}
            speaking = re.split('[<>]', line)
            
            curspeech["speaker"] = speaking[6]
            curspeech["link"] = link
            curspeech["date"] = date
            curspeech["speech"] = html.document_fromstring(line).text_content()[len(speaking[6])+1:].strip()
        elif "<a href=" in line[:30] and "hlasy.sqw" not in line and "historie.sqw" not in line:
            if speaking: 
                output.append(curspeech)
                curspeech = {}
            speaking = re.split('[<>]',line)
            curspeech["speaker"] = speaking[4]
            curspeech["link"] = link
            curspeech["date"] = date
            curspeech["speech"] = speaking[6][1:].strip()
        elif "<b><u>" in line:
            if speaking: 
                output.append(curspeech)
                curspeech = {}
            speaking = re.split('[<>]',line)
            
            curspeech["speaker"] = speaking[6]
            curspeech["link"] = link
            curspeech["date"] = date
            curspeech["speech"] = speaking[10][1:].strip()
        elif speaking:
            try: line = html.document_fromstring(line).text_content()
            except:
                line += "<div></div>"
                line = html.document_fromstring(line).text_content()
            if line.strip() != "": curspeech["speech"] += " " + line.strip()
    output.append(curspeech)
    return output

In [116]:
os.makedirs("json", exist_ok=True)

for year in ["2010", "2013", "2017"]:
    os.makedirs("json/"+year, exist_ok=True)
    for folder in os.listdir(year):
        if "schuz" in folder:
            merged_folder = proc_merged(year+"/"+folder)
            with open("json/"+year+"/"+folder+".json", "w", encoding="utf8") as file:
                json.dump(merged_folder, file, ensure_ascii=False)